# Sorting, Graph Paths, Dynamic Programming
Objectives:
- In-place quicksort vs heapsort vs nth_element
- Dijkstra vs A* vs BFS on a grid
- DP patterns: LIS, knapsack, edit distance

In [ ]:
# Starter imports
import heapq
import random
from pathlib import Path

# TODO: implement sort benchmarks, graph searches, and DP examples

In [ ]:
import heapq, random, timeit

def quicksort_inplace(arr, lo=0, hi=None):
    if hi is None: hi = len(arr) - 1
    if lo >= hi: return
    pivot = arr[(lo + hi)//2]
    i, j = lo, hi
    while i <= j:
        while arr[i] < pivot: i += 1
        while arr[j] > pivot: j -= 1
        if i <= j:
            arr[i], arr[j] = arr[j], arr[i]
            i += 1; j -= 1
    quicksort_inplace(arr, lo, j)
    quicksort_inplace(arr, i, hi)

def bench_sorts(n=50_000):
    data = [random.randint(0, 1_000_000) for _ in range(n)]
    cases = {
        "quicksort": lambda: quicksort_inplace(data.copy()),
        "sorted_builtin": lambda: sorted(data),
        "heapsort": lambda: [heapq.heappop(heapq.heapify(list(data)) or []) for _ in range(len(data))],
        "nth_element": lambda: heapq.nsmallest(10, data),
    }
    return {k: timeit.timeit(v, number=1) for k, v in cases.items()}

bench_sorts(20_000)

In [ ]:
from heapq import heappush, heappop

def grid_neighbors(r, c, rows, cols):
    for dr, dc in ((1,0),(-1,0),(0,1),(0,-1)):
        nr, nc = r+dr, c+dc
        if 0 <= nr < rows and 0 <= nc < cols:
            yield nr, nc

def bfs_grid(rows=5, cols=5, start=(0,0), goal=(4,4)):
    from collections import deque
    q = deque([start]); seen = {start}; parent = {}
    while q:
        r, c = q.popleft()
        if (r, c) == goal: break
        for nr, nc in grid_neighbors(r, c, rows, cols):
            if (nr, nc) not in seen:
                seen.add((nr, nc))
                parent[(nr, nc)] = (r, c)
                q.append((nr, nc))
    return parent

def dijkstra_grid(costs, start, goal):
    rows, cols = len(costs), len(costs[0])
    pq = [(0, start)]; dist = {start: 0}; parent = {}
    while pq:
        d, node = heappop(pq)
        if node == goal: break
        if d != dist[node]: continue
        for nbr in grid_neighbors(*node, rows, cols):
            nd = d + costs[nbr[0]][nbr[1]]
            if nd < dist.get(nbr, float('inf')):
                dist[nbr] = nd
                parent[nbr] = node
                heappush(pq, (nd, nbr))
    return parent, dist.get(goal)

def astar_grid(costs, start, goal):
    rows, cols = len(costs), len(costs[0])
    def h(pos):
        return abs(pos[0]-goal[0]) + abs(pos[1]-goal[1])
    pq = [(0, start)]; g = {start: 0}; parent = {}
    while pq:
        f, node = heappop(pq)
        if node == goal: break
        for nbr in grid_neighbors(*node, rows, cols):
            tentative = g[node] + costs[nbr[0]][nbr[1]]
            if tentative < g.get(nbr, float('inf')):
                g[nbr] = tentative
                parent[nbr] = node
                heappush(pq, (tentative + h(nbr), nbr))
    return parent, g.get(goal)

costs = [[1]*5 for _ in range(5)]
parent_bfs = bfs_grid()
parent_dij, dist_dij = dijkstra_grid(costs, (0,0), (4,4))
parent_astar, dist_astar = astar_grid(costs, (0,0), (4,4))
dist_dij, dist_astar

In [ ]:
def lis(seq):
    import bisect
    tails = []
    for x in seq:
        i = bisect.bisect_left(tails, x)
        if i == len(tails): tails.append(x)
        else: tails[i] = x
    return len(tails)

def knapsack(weights, values, capacity):
    n = len(weights)
    dp = [[0]*(capacity+1) for _ in range(n+1)]
    for i in range(1, n+1):
        w, v = weights[i-1], values[i-1]
        for c in range(capacity+1):
            dp[i][c] = dp[i-1][c]
            if w <= c:
                dp[i][c] = max(dp[i][c], v + dp[i-1][c-w])
    return dp[n][capacity]

def edit_distance(a: str, b: str):
    dp = [[0]*(len(b)+1) for _ in range(len(a)+1)]
    for i in range(len(a)+1): dp[i][0] = i
    for j in range(len(b)+1): dp[0][j] = j
    for i in range(1, len(a)+1):
        for j in range(1, len(b)+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,
                dp[i][j-1] + 1,
                dp[i-1][j-1] + cost,
            )
    return dp[-1][-1]

lis([3,1,5,2,6])
knapsack([2,3,4], [4,5,6], 5)
edit_distance("kitten", "sitting")

Notes:
- Quicksort in-place expected O(n log n), heapsort O(n log n), nth_element ~ O(n log k).
- BFS shortest paths on unweighted graphs; Dijkstra for weighted; A* uses heuristic to guide search.
- DP examples show classic table + n log n LIS with patience sorting.